<a href="https://colab.research.google.com/github/Adhambarakat/supreme-chainsaw/blob/main/regressionXiot999.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Dependencies

In [1]:
!pip install influxdb-client
!pip install flightsql-dbapi
!pip install pandas
!pip3 install paho-mqtt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.2/721.2 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.8/217.8 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.0 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.18
    Uninstalling SQLAlchemy-2.0.18:
      Successfully uninstalled SQLAlchemy-2.0.18
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 9.0.0
    Uninstalling pyarrow-9.0.0:
      Successfully uninstalled pyarrow-9.0.0
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. T

Importing Libraries

In [2]:
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
import time
import paho.mqtt.client as paho
from paho import mqtt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.engine.training import optimizer
from google.colab import files
import matplotlib.pyplot as plt
import keras.utils as to_categorica

Loading Data

In [ ]:
path = files.upload()

Saving ppgsbpdbp.xlsx to ppgsbpdbp.xlsx


Allocating Data

In [13]:
data = pd.read_excel('/content/a7a6.xlsx')
input_dim=1
x_train = data['PPG']  #ptt training
systolic = data['SBP']  #systolic
diastolic = data['DBP'] #diastolic
y_train = np.column_stack((systolic,diastolic))

Defining the Model

In [118]:
#defining the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=16, activation='relu', input_shape=[input_dim]),
    tf.keras.layers.Dense(units=2, input_shape=[input_dim])
])

Compiling the Model

In [119]:
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mae'])

Training the Model

In [ ]:
model.fit(abs(x_train), y_train, epochs=10, batch_size=10)

Epoch 1/10
104858/104858 [==============================] - 175s 2ms/step - loss: 304.3139 - mae: 10.1932
Epoch 2/10
104858/104858 [==============================] - 176s 2ms/step - loss: 116.0851 - mae: 8.0251
Epoch 3/10
104858/104858 [==============================] - 175s 2ms/step - loss: 116.0914 - mae: 8.0252
Epoch 4/10
104858/104858 [==============================] - 175s 2ms/step - loss: 116.0913 - mae: 8.0254
Epoch 5/10
104858/104858 [==============================] - 176s 2ms/step - loss: 116.0910 - mae: 8.0252
Epoch 6/10
 13188/104858 [==>...........................] - ETA: 2:22 - loss: 116.2480 - mae: 8.0218

Connecting to Influxdb

In [103]:
# !export INFLUXDB_TOKEN=-g6iC2cK-mwZpYCPaiKdDOyyX5P1vkncIDSigkxs_ImaQJUDUm1-87FX4peKEmW4F2DC9CMSqZKVVTLNvXERSg==

#token = os.environ.get("INFLUXDB_TOKEN")
token = "-g6iC2cK-mwZpYCPaiKdDOyyX5P1vkncIDSigkxs_ImaQJUDUm1-87FX4peKEmW4F2DC9CMSqZKVVTLNvXERSg=="
org = "test"
url = "https://us-east-1-1.aws.cloud2.influxdata.com"

write_client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)

x = 123
print (x)

123


Receiving and Sending Data

In [108]:
# setting callbacks for different events to see if it works, print the message etc.
def on_connect(client, userdata, flags, rc, properties=None):
    print("CONNACK received with code %s." % rc)

# with this callback you can see if your publish was successful
def on_publish(client, userdata, mid, properties=None):
    print("mid: " + str(mid))

# print which topic was subscribed to
def on_subscribe(client, userdata, mid, granted_qos, properties=None):
    print("Subscribed: " + str(mid) + " " + str(granted_qos))

# print message, useful for checking if it was successful
def on_message(client, userdata, msg):
    #print(msg.topic + " " + str(msg.qos) + " " + str(msg.payload))
    #print(int(msg.payload))
    if msg.topic == "avgbpm" :
      bucket="colabtest"
      # Define the write api
      write_api = write_client.write_api(write_options=SYNCHRONOUS)
      data = {
        "point1": {
          "location": "Hospital",
          "dataa": "avghr",
          "valuee": int(msg.payload),
        },
      }
      for key in data:
        point = (
          Point("avghr")
          .tag("location", data[key]["location"])
          .field(data[key]["dataa"], data[key]["valuee"])
        )
      write_api.write(bucket=bucket, org=org, record=point)
      time.sleep(1) # separate points by 1 second
      print("Complete. Return to the InfluxDB UI.")
      print(msg.payload)
    elif msg.topic == "ppg" :
      x_new=[[float(int(msg.payload)/100)]]
      y_new = model.predict(np.array([[x_new]]))
      a = y_new[0]
      b = a[0]
      c = b[0]
      sbp = int(c[0])
      dbp = int(c[1])
      bucket="colabtest"
      # Define the write api
      write_api = write_client.write_api(write_options=SYNCHRONOUS)
      data = {
        "point2": {
          "location": "Hospital",
          "dataa": "sbp",
          "valuee": sbp,
        },
      }
      for key in data:
        point = (
          Point("sp")
          .tag("location", data[key]["location"])
          .field(data[key]["dataa"], data[key]["valuee"])
        )
      write_api.write(bucket=bucket, org=org, record=point)
      time.sleep(1) # separate points by 1 second
      print("Complete. Return to the InfluxDB UI.")
      bucket="colabtest"
      # Define the write api
      write_api = write_client.write_api(write_options=SYNCHRONOUS)
      data = {
        "point3": {
          "location": "Hospital",
          "dataa": "dbp",
          "valuee": dbp+20,
        },
      }
      for key in data:
        point = (
          Point("dp")
          .tag("location", data[key]["location"])
          .field(data[key]["dataa"], data[key]["valuee"])
        )
      write_api.write(bucket=bucket, org=org, record=point)
      time.sleep(1) # separate points by 1 second
      print("Complete. Return to the InfluxDB UI.")
      print(sbp)
      print(dbp+20)
    else :
      bucket="colabtest"
      # Define the write api
      write_api = write_client.write_api(write_options=SYNCHRONOUS)
      data = {
        "point4": {
          "location": "Hospital",
          "dataa": "acchr",
          "valuee": int(msg.payload),
        },
      }
      for key in data:
        point = (
          Point("acchr")
          .tag("location", data[key]["location"])
          .field(data[key]["dataa"], data[key]["valuee"])
        )
      write_api.write(bucket=bucket, org=org, record=point)
      time.sleep(1) # separate points by 1 second
      print("Complete. Return to the InfluxDB UI.")
      print(msg.payload)


    """
    bucket="colabtest"
    # Define the write api
    write_api = write_client.write_api(write_options=SYNCHRONOUS)
    data = {
      "point4": {
        "location": "Hospital",
        "dataa": "hr",
        "valuee": int(msg.payload),
      },
    }
    for key in data:
      point = (
        Point("bps")
        .tag("location", data[key]["location"])
        .field(data[key]["dataa"], data[key]["valuee"])
      )
    write_api.write(bucket=bucket, org=org, record=point)
    time.sleep(1) # separate points by 1 second
    print("Complete. Return to the InfluxDB UI.")
    """



# using MQTT version 5 here, for 3.1.1: MQTTv311, 3.1: MQTTv31
# userdata is user defined data of any type, updated by user_data_set()
# client_id is the given name of the client
client = paho.Client(client_id="", userdata=None, protocol=paho.MQTTv5)
client.on_connect = on_connect

# enable TLS for secure connection
client.tls_set(tls_version=mqtt.client.ssl.PROTOCOL_TLS)
# set username and password
client.username_pw_set("ahmed", "Ahmed1603")
# connect to HiveMQ Cloud on port 8883 (default for MQTT)
client.connect("7ea331cfad864375a4a189652c9375e0.s1.eu.hivemq.cloud", 8883)

# setting callbacks, use separate functions like above for better visibility
client.on_subscribe = on_subscribe
client.on_message = on_message
client.on_publish = on_publish

# subscribe to all topics of encyclopedia by using the wildcard "#"
client.subscribe("bpm", qos=1)
client.subscribe("ppg", qos=1)
client.subscribe("avgbpm", qos=1)

# a single publish, this can also be done in loops, etc.
# client.publish("encyclopedia/temperature", payload="hot", qos=1)

# loop_forever for simplicity, here you need to stop the loop manually
# you can also use loop_start and loop_stop
client.loop_forever()

CONNACK received with code Success.
Subscribed: 1 [<paho.mqtt.reasoncodes.ReasonCodes object at 0x7f78226760b0>]
Subscribed: 2 [<paho.mqtt.reasoncodes.ReasonCodes object at 0x7f7822676050>]
Subscribed: 3 [<paho.mqtt.reasoncodes.ReasonCodes object at 0x7f7822676140>]
Complete. Return to the InfluxDB UI.
b'67'
1/1 [==============================] - 0s 82ms/step
Complete. Return to the InfluxDB UI.
Complete. Return to the InfluxDB UI.
125
69
Complete. Return to the InfluxDB UI.
b'62'
1/1 [==============================] - 0s 34ms/step
Complete. Return to the InfluxDB UI.
Complete. Return to the InfluxDB UI.
126
68
Complete. Return to the InfluxDB UI.
b'59'
1/1 [==============================] - 0s 19ms/step
Complete. Return to the InfluxDB UI.
Complete. Return to the InfluxDB UI.
124
70
Complete. Return to the InfluxDB UI.
b'67'
Complete. Return to the InfluxDB UI.
b'2'
1/1 [==============================] - 0s 32ms/step
Complete. Return to the InfluxDB UI.
Complete. Return to the InfluxD

KeyboardInterrupt: ignored